# Constrained MDP (CMDP)
---

今回はCMDPを考えたアルゴリズムの中でもOpt_CMDPというアルゴリズムを考えます．



In [12]:
# 環境の定義

import numpy as np
import pickle

states = np.arange(0,3)
actions = {0:[0,1],1:[0,1],2:[0,1]} # 0:遷移しない, 1:遷移する

S = len(states)
A = 2

#遷移確率と報酬とコストを定義
P  = np.zeros((S, A, S))
R = np.zeros((S, A))
C = np.zeros((S, A))

for s in range(S):
    for a in range(A):

        if a == 0:
            P[s,a,s] = 1
        else:
            R[s,a] = -(s + 1)
            C[s,a] = 1

            if s < S-1:
                P[s,a,s+1] = 1
            else:
                P[s,a,0] = 1
             
#報酬の絶対値の大きさを1以下にする
for s in range(S):
    for a in range(A):
        R[s][a] = R[s][a]/3

print(R)
print(C)
print(P)

states_abs = np.arange(0,1)
nA_abs = 2


N_STATES_abs = len(states_abs)

P_abs = np.zeros((1,2,1))
R_abs = np.zeros((1,2))
C_abs = np.zeros((1,2))

P_abs[0][0][0] = 1
P_abs[0][1][0] = 1

R_abs[0][1] = 2

C_abs[0][1] = 1



[[ 0.         -0.33333333]
 [ 0.         -0.66666667]
 [ 0.         -1.        ]]
[[0. 1.]
 [0. 1.]
 [0. 1.]]
[[[1. 0. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [1. 0. 0.]]]


In [16]:
EPISODE_LENGTH = 6
CONSTRAINT = EPISODE_LENGTH/2
C_b = CONSTRAINT/5
NUMBER_EPISODES = 1e5
NUMBER_SIMULATIONS = 1

eps = 0.01
M = 0
delta = 0.01

P_hat = np.zeros((S, A, S))
R_hat = np.zeros((S, A))
C_hat = np.zeros((S, A))
Total_emp_reward = np.zeros((S, A))
Total_emp_cost = np.zeros((S ,A))

s_a_visit = np.zeros((S, A))
s_a_s_visit = np.zeros((S, A, S))
beta_prob = np.zeros((S,A,S))
beta_prob_2 = np.zeros((S,A,S))
beta_r = np.zeros((S,A))
beta_prob_T = np.zeros((S,A,S))

mu = np.zeros((S))
mu[0] = 1


In [15]:
#次の状態や報酬を返す関数   
def step(state,action):
    next_state = int(np.random.choice(np.arange(S),1,replace=True,p=P[state,action,:]))
    rew = R[state,action]
    cost = C[state,action]
    return next_state,rew,cost

#訪問回数を更新する関数
def setCounts(s_a_s_count,s_a_count):
    for s in range(S):
        for a in range(A):
            s_a_visit[s,a] += s_a_count[s,a]
            for s_ in range(S):
                s_a_s_visit[s,a,s_] += s_a_s_count[s,a,s_]

#経験的な遷移関数の更新
def update_P_hat():
    for s in range(S):
        for a in range(A):

            if s_a_visit[s][a] == 0:
                P_hat[s][a] = 1 / S * np.ones(S)
            
            else:
                for s_ in range(S):
                    P_hat[s][a][s_] = s_a_s_visit[s][a][s_] / s_a_visit[s][a]
                
                P_hat[s][a] = P_hat[s][a] / np.sum(P_hat[s][a])

#経験的な報酬とコストの更新
def update_RC_hat(emp_r,emp_c):
    for s in range(S):
        for a in range(A):
            R_hat[s][a] = (Total_emp_reward[s][a] + emp_r[s][a]) / (max(s_a_visit[s][a],1))
            C_hat[s][a] = (Total_emp_cost[s][a] + emp_c[s][a]) / (max(s_a_visit[s][a],1))

            Total_emp_reward += emp_r
            Total_emp_cost += emp_c

#信頼区間の計算
def calc_confidence_interval():
    for s in range(S):
        for a in range(A):
            beta_r[s][a] = np.sqrt(1/max(1,s_a_visit[s][a]))
            for s_ in range(S):
                beta_prob[s][a][s_] = np.sqrt(P_hat[s][a][s_]*(1-P_hat[s][a][s_])/max(s_a_visit[s][a],1)) + 1/(max(s_a_visit[s][a],1))




In [17]:
# 価値関数やコスト関数の計算
def Policy_evaluation(P,policy,R,C):
    Q_h = np.zeros((S,EPISODE_LENGTH,A))
    V_h = np.zeros((S,EPISODE_LENGTH))
    C_h = np.zeros((S,EPISODE_LENGTH))

    P_policy = np.zeros((S,EPISODE_LENGTH,S))
    R_policy = np.zeros((S,EPISODE_LENGTH))
    C_policy = np.zeros((S,EPISODE_LENGTH))


    
    for s in range(S):
        for a in range(A):
            C_h[s][EPISODE_LENGTH-1] += policy[s][EPISODE_LENGTH-1][a] * C[s][a]
            Q_h[s,EPISODE_LENGTH-1,a] = R[s][a]

        V_h[s,EPISODE_LENGTH-1] = np.dot(Q_h[s,EPISODE_LENGTH-1,:],policy[s,EPISODE_LENGTH-1,:])
    
    for h in range(EPISODE_LENGTH):
        for s in range(S):
            for a in range(A):
                C_policy[s][h] += policy[s][h][a] * C[s][a]
            
            for s_ in range(S):
                for a in range(A):
                    P_policy[s,h,s_] += policy[s][h][a] * P[s][a][s_]
    
    for h in range(EPISODE_LENGTH,-1,-1): #0~4
        for s in range(S):
            C_h[s,h] = C_policy[s,h] + np.dot(P_policy[s,h,:],C_h[:,h+1])
            for a in range(A):
                P_V = 0
                for s_ in range(S):
                    P_v += P_policy[s,h,s_] * V_h[s_,h+1]
                Q_h[s,h,a] = R[s][a] + P_V
            V_h[s,h] = np.dot(Q_h[s,h,:],policy[s,h,:])
    
    return Q_h,V_h,C_h


    

                
    
        

        
        
    

---

ここからExtended LP問題を考えるのですが，この[サイト](https://qiita.com/ytakashina/items/9de38882ffa611d5a07a)からpulpの大きいモデルではnumpy.sumを避ける理由が書いてあります．


In [14]:
# CMDPを解くための線形計画法
import pulp
import math

def solve_CMDP():
    policy = np.zeros((S,EPISODE_LENGTH,A))
    problem = pulp.LpProblem('CMDP', pulp.LpMinimize)
    z_opt = np.zeros((EPISODE_LENGTH,S,A,S))
    z_keys = [(h,s,a,s_) for h in range(EPISODE_LENGTH) for s in range(S) for a in range(A) for s_ in range(S)]
    z = pulp.LpVariable.dicts('z_var',z_keys,0,1,cat='Continuous')
    
    #最適したい問題
    problem += pulp.lpSum([z[(h,s,a,s_)] * R[s][a] for h in range(EPISODE_LENGTH) for s in range(S) for a in range(A) for s_ in range(S)])

    #制約条件
    problem += pulp.lpSum([z[(h,s,a,s_)] * C[s][a] for h in range(EPISODE_LENGTH) for s in range(S) for a in range(A) for s_ in range(S)]) <= CONSTRAINT

    for h in range(1,EPISODE_LENGTH):
        for s in range(S):
            z_list = [z[(h,s,a,s_)] for a in range(A) for s_ in range(S)]
            z_prev_list = [z[(h-1,s_,a,s)] for a in range(A) for s_ in range(S)]
            problem += pulp.lpSum(z_list) == pulp.lpSum(z_prev_list)

    for s in range(S):
        z_1_lsit = [z[(0,s,a,s_)] for a in range(A) for s_ in range(S)]
        problem += pulp.lpSum(z_1_lsit) == mu[s]
    
    problem += [z[(h,s,a,s_)] for h in range(EPISODE_LENGTH) for s in range(S) for a in range(A) for s_ in range(S)] >= 0

    for h in range(EPISODE_LENGTH):
            for s in range(S):
                for a in range(A):
                    for s_1 in range(S):
                        opt_prob += z[(h,s,a,s_1)] - (P_hat[s][a][s_1] + beta_prob[s][a,s_1]) *  pulp.lpSum([z[(h,s,a,y)] for y in range(S)]) <= 0
                        opt_prob += -z[(h,s,a,s_1)] + (P_hat[s][a][s_1] - beta_prob[s][a,s_1])* pulp.lpSum([z[(h,s,a,y)] for y in range(S)]) <= 0

    #問題を解く
    status = problem.solve(pulp.PULP_CBC_CMD(msg=0,fracGap=0.0001))

    if pulp.LpStatus[status] != 'Optimal':
        print('Not Optimal')
        return np.zeros((S,EPISODE_LENGTH,S)),np.zeros((S,EPISODE_LENGTH)),np.zeros((S, EPISODE_LENGTH)), pulp.LpStatus[status], np.zeros((S, EPISODE_LENGTH,S))
    
    for h in range(EPISODE_LENGTH):
        for s in range(S):
            for a in range(A):
                for s_ in range(S):
                    z_opt[h][s][a][s_] = z[(h,s,a,s_)].varValue

    z_s_a = np.sum(z_opt,axis=3)
    z_s = np.sum(z_opt,axis=(2,3))

    for h in range(EPISODE_LENGTH):
        for s in range(S):
            sum_probs = 0
            for a in range(A):
                policy[s,h,a] = z_s_a[h][s][a] / z_s[h][s]
                sum_probs += policy[s,h,a]
            
            if sum_probs.sum(axis=(0,1,2)) == 0:
                for a in range(A):
                    policy[s,h,a] = 1 / A
            
            else:
                for a in range(A):
                    policy[s,h,a] = policy[s,h,a] / sum_probs
    
    q_policy, value_of_policy, cost_of_policy = FiniteHorizon_Policy_evaluation(P, policy,R,C)

    return policy,value_of_policy,cost_of_policy, pulp.LpStatus[status], q_policy


            




array([0.33333333, 0.33333333, 0.33333333])